In [67]:
"""Name: Muhammad Hashir
Date: 09/90/25"""


'Name: Muhammad Hashir\nDate: 09/90/25'

In [142]:
import pandas as pd 
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [143]:
mean_age = df['Age'].mean()
df['Age'].fillna(mean_age, inplace=True)

# 2. Impute Embarked: Fill with the mode (most common port)
mode_embarked = df['Embarked'].mode()[0]
df['Embarked'].fillna(mode_embarked, inplace=True)

# --- Store Training Statistics for Test Set Use (Fixes NameError) ---
train_mean_age = mean_age
train_mode_embarked = mode_embarked

# --- Feature Dropping and Separation ---
df.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True, errors='ignore')
y = df['Survived']
X = df.drop(columns=['Survived', 'PassengerId'])

/var/folders/m3/8kjpfvdd6td95pxz_k210f6m0000gn/T/ipykernel_72231/4267558607.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(mean_age, inplace=True)
/var/folders/m3/8kjpfvdd6td95pxz_k210f6m0000gn/T/ipykernel_72231/4267558607.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behav

In [144]:
realcols = ["Age","Pclass","SibSp", "Parch", "Fare"]
train_scaling_stats = {}

for col in realcols:
  mean = X[col].mean()
  std = X[col].std()
  
  # Store the mean and std for the test set
  train_scaling_stats[col] = {'mean': mean, 'std': std}
  
  # Apply scaling to the training set X
  X[col] = (X[col] - mean) / std
  
traincol = X.columns.tolist() # This is now defined *after* scaling, but is still not the final feature list.

In [145]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split

X = pd.get_dummies(X, columns=['Sex', 'Embarked'], drop_first=True) 

train_cols_final = X.columns.tolist() 

X = X.fillna(X.mean())


X_train, y_train = X, y
logisticReg = LogisticRegression()
logisticReg.fit(X_train, y_train)

print(f"Model trained successfully with {X_train.shape[0]} samples and {X_train.shape[1]} features.")


Model trained successfully with 891 samples and 8 features.


In [146]:
df_test = pd.read_csv('test.csv')
test_ids = df_test['PassengerId'] 

# Dropping unnecessary columns
df_test.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True, errors="ignore")

# 1. Impute Test Data using TRAIN SET Statistics
df_test['Age'].fillna(train_mean_age, inplace=True)
df_test['Embarked'].fillna(train_mode_embarked, inplace=True)

# The 'Fare' column is missing for one row in the test set. 
# We need to impute it using the *training set's* fare mean.
train_fare_mean = train_scaling_stats['Fare']['mean']
df_test['Fare'].fillna(train_fare_mean, inplace=True)


X_test = df_test.drop(columns=['PassengerId']) 

# 2. Scale Test Data using TRAIN SET Statistics
realcols = ["Age","Pclass","SibSp", "Parch", "Fare"]
for col in realcols:
  mean = train_scaling_stats[col]['mean']
  std = train_scaling_stats[col]['std']
  
  # Apply scaling
  X_test[col] = (X_test[col] - mean) / std if std != 0 else X_test[col] - mean 

# 3. Perform the one-hot encoding on the test set
X_test = pd.get_dummies(X_test, columns=['Sex', 'Embarked'], drop_first=True) 

# 4. Reindex using the correct list of feature names from the trained model (train_cols_final)
X_test = X_test.reindex(columns=train_cols_final, fill_value=0)

/var/folders/m3/8kjpfvdd6td95pxz_k210f6m0000gn/T/ipykernel_72231/3789877998.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test['Age'].fillna(train_mean_age, inplace=True)
/var/folders/m3/8kjpfvdd6td95pxz_k210f6m0000gn/T/ipykernel_72231/3789877998.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

In [147]:
y_pred_survived = logisticReg.predict(X_test)
submission_df = pd.DataFrame({
    'PassengerId': test_ids,
    'Survived': y_pred_survived.astype(int)
})

print("Submission Data Format:")
print(submission_df.head())

submission_df.to_csv('submission_predictions.csv', index=False)

Submission Data Format:
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1
